# Contents

* using mlflow to log parameters

# Setup

In [50]:
import tempfile
import json
import os 
import logging
logging.getLogger().setLevel(logging.INFO)

import mlflow
import pandas as pd
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, r2_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [53]:
logging.info("C")

INFO:root:C


# Raw dataset

In [10]:
filename = '../../data/raw/tweets-train.csv'
df = pd.read_csv(filename)

In [11]:
df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


# Removing nan cells

In [12]:
df = df.dropna()

# Model

In [103]:
# variables
X = df['text']
Y = df['sentiment']
# Train test split
X_train, x_test, Y_train, y_test = train_test_split(X, Y, random_state=42)

with mlflow.start_run(run_name='suport_vector_clasifier'):
    mlflow.log_param('file name', filename.split("/")[-1])
    try:
        
        count_vectorizer = CountVectorizer(stop_words='english', strip_accents='ascii', lowercase=True, max_features=25000)
        svm = SVC(probability=True)
    except ValueError: 
        logging.log(40," Error encountered in model instanciation.".format(item))
    
    logging.info(" Model instances created")
    # creating a pipeline
    pipe = Pipeline(steps = [('vectorizer', count_vectorizer),
                             ('svm_model', svm)])
    
    logging.info(" Pipeline created.")
    
    pipe.fit(X_train, Y_train)
    log_params = [mlflow.log_param(key, value) for key, value in pipe.get_params().items() if '__' in key]
    logging.info(" Logged params.")
    
    # Getting predicions 
    predictions = pipe.predict(x_test)
    logging.info(" Predictions out.")
    
    # metrics 
    metrics_ = {'accuracy': accuracy_score,
                'balanced_accuracy': metrics.balanced_accuracy_score,
              #  'average_precision': metrics.average_precision_score,
              #  'precission': metrics.precision_score,
                'recall': metrics.recall_score,
               }
    
    
    for metric, func in metrics_.items():
        try:
            result = func(y_test, predictions)
            print(f"{metric} = {result}")
            mlflow.log_metric(metric, result)
        except ValueError:
            logging.log(40, f"Not logged metric {metric}.")
            pass
        
    
    logging.info(" Metrics logged.")

INFO:root: Model instances created
INFO:root: Pipeline created.
INFO:root: Logged params.
INFO:root: Predictions out.
ERROR:root:Not logged metric recall.
INFO:root: Metrics logged.


accuracy = 0.6899563318777293
balanced_accuracy = 0.6738988499738578


# GRidSearch

In [5]:
# spliting data
X = df['text']
Y = df['sentiment']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [15]:
# creating the pipeline
vectorizer = CountVectorizer(stop_words='english', strip_accents='ascii', lowercase=True, max_features=23000)
svc = SVC()
pipe = Pipeline(steps = [('vectorizer', vectorizer),
                         ('classifier', svc)])

In [16]:
# Creating grid
param_grid = {'classifier__C':[0.1, 1, 10, 100],
              'classifier__gamma':[1,0.1,0.01,0.001],
              'classifier__kernel': ['rbf', 'poly', 'sigmoid']}
# cv = 5 by default
gsearch = GridSearchCV(pipe, param_grid=param_grid, refit=True, cv=2, verbose=-1, n_jobs=-1)
gsearch.fit(X_train, Y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed: 20.8min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vectorizer',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=23000,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                     

In [23]:
print("Best parameter (CV score=%0.3f):" % gsearch.best_score_)
print("Best Parametres: ", gsearch.best_params_)
print("Best Estimator ", gsearch.best_estimator_)

Best parameter (CV score=0.680):
Best Parametres:  {'classifier__C': 10, 'classifier__gamma': 0.1, 'classifier__kernel': 'rbf'}
Best Estimator  Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=23000, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents='ascii',
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 SVC(C=10, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', de

In [30]:
pd.DataFrame(gsearch.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_classifier__gamma,param_classifier__kernel,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,34.848027,0.090948,19.255964,0.043975,0.1,1,rbf,"{'classifier__C': 0.1, 'classifier__gamma': 1,...",0.414015,0.413392,0.413703,0.000312,35
1,38.059688,4.395980,14.929943,0.002498,0.1,1,poly,"{'classifier__C': 0.1, 'classifier__gamma': 1,...",0.599709,0.604596,0.602152,0.002443,14
2,22.549576,0.143418,12.659245,0.089449,0.1,1,sigmoid,"{'classifier__C': 0.1, 'classifier__gamma': 1,...",0.635579,0.631940,0.633760,0.001820,12
3,31.220106,0.230869,18.220058,0.011493,0.1,0.1,rbf,"{'classifier__C': 0.1, 'classifier__gamma': 0....",0.506550,0.507902,0.507226,0.000676,28
4,30.414068,0.073457,16.928298,0.131925,0.1,0.1,poly,"{'classifier__C': 0.1, 'classifier__gamma': 0....",0.411104,0.410896,0.411000,0.000104,36
5,25.615819,0.065462,16.122262,0.053471,0.1,0.1,sigmoid,"{'classifier__C': 0.1, 'classifier__gamma': 0....",0.532855,0.535038,0.533947,0.001092,26
6,31.618378,0.015490,18.645815,0.140420,0.1,0.01,rbf,"{'classifier__C': 0.1, 'classifier__gamma': 0....",0.427948,0.429507,0.428727,0.000780,34
7,24.056211,0.140921,16.212209,0.007495,0.1,0.01,poly,"{'classifier__C': 0.1, 'classifier__gamma': 0....",0.407673,0.407673,0.407673,0.000000,40
8,27.156435,0.180897,16.367120,0.015491,0.1,0.01,sigmoid,"{'classifier__C': 0.1, 'classifier__gamma': 0....",0.410064,0.409025,0.409545,0.000520,38
9,30.156715,0.182896,18.960634,0.067461,0.1,0.001,rbf,"{'classifier__C': 0.1, 'classifier__gamma': 0....",0.407673,0.407673,0.407673,0.000000,40


In [33]:
predictions = gsearch.predict(X_test)

In [36]:
accuracy = accuracy_score(Y_test, predictions)
accuracy

0.7009946627850558

In [40]:
print(classification_report(Y_test, predictions)

              precision    recall  f1-score   support

    negative       0.73      0.63      0.68      2390
     neutral       0.65      0.73      0.69      3275
    positive       0.76      0.73      0.74      2579

    accuracy                           0.70      8244
   macro avg       0.71      0.70      0.70      8244
weighted avg       0.71      0.70      0.70      8244



In [ ]:
gsearch.

In [47]:
labels=['positive', 'neutral', 'negative']
pd.DataFrame(confusion_matrix(results['y'],results['y_hat'],labels=labels), 
             index= labels, 
             columns=labels)

,positive,neutral,negative
positive,1874,587,118
neutral,429,2398,448
negative,155,728,1507


In [51]:
labels=['positive', 'neutral', 'negative']
pd.DataFrame(confusion_matrix(results['y'],results['y_hat'],labels=labels, normalize='true'), 
             index= labels, 
             columns=labels)

,positive,neutral,negative
positive,0.726638,0.227608,0.045754
neutral,0.130992,0.732214,0.136794
negative,0.064854,0.304603,0.630544


# Ploting learning curves

In [5]:
from sklearn.model_selection import learning_curve
import numpy as np
import matplotlib.pyplot as plt

In [6]:

def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    axes : array of 3 axes, optional (default=None)
        Axes to use for plotting the curves.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [7]:
plot_learning_curve(gsearch.best_estimator_, "title", X_train, Y_train, axes=axes[:, 0], ylim=(0.7, 1.01),
                    cv=2, n_jobs=4)

NameError: name 'gsearch' is not defined

In [ ]:
 sklearn.metrics.plot_roc_curve(estimator, X, y, *, sample_weight=None, drop_intermediate=True, response_method='auto', name=None, ax=None, **kwargs)[source]